In [ ]:
s3_bucket = '<INSERT S3 BUCKET HERE>'

In [ ]:
prefix = 'pipeline'

In [ ]:
endpoint_name = "AutoGluonEndpoint"

In [ ]:
from sagemaker.predictor import Predictor

import sagemaker

In [ ]:
session = sagemaker.Session()

In [ ]:
!mkdir -p tmp

In [ ]:
test_data_path = f"s3://{s3_bucket}/{prefix}/output/test/data.csv"

In [ ]:
!aws s3 cp {test_data_path} tmp/test_data.csv

In [ ]:
import pandas as pd

test_df = pd.read_csv("tmp/test_data.csv", header=None)
test_df

In [ ]:
test_df.rename(
    columns={ 
        test_df.columns[0]: "is_cancelled" 
    }, 
    inplace = True
)

In [ ]:
test_df

In [ ]:
predictor = Predictor(endpoint_name, session)

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [ ]:
import csv

def get_test_payload(index, test_df=test_df):
    test_data = test_df.drop(['is_cancelled'], axis=1)
    target_record = test_data.iloc[index]
    predictor_values = target_record.to_csv(
        header=None,
        index=False,
        quotechar='"',
        quoting=csv.QUOTE_NONNUMERIC
    ).split()
    csv_string = ','.join(predictor_values)
    return csv_string

In [ ]:
def get_test_actual_result(index, test_df=test_df):
    result = test_df.iloc[index]['is_cancelled']
    return result

In [ ]:
def predict(index, predictor=predictor):
    payload = get_test_payload(index)
    prediction = predictor.predict([payload])
    print(prediction)
    [[prob_0, prob_1]] = prediction['probabilities']
    
    if prob_0 > prob_1:
        return 0
    else:
        return 1

In [ ]:
predict(5)

In [ ]:
from time import sleep

actual_list = []
predicted_list = []

for i in range(0, 100):
    actual = get_test_actual_result(i)
    predicted = predict(i)
    print(f"[iteration # {i}]")
    print(f"actual = {actual}; predicted = {predicted}")
    
    actual_list.append(actual)
    predicted_list.append(predicted)

    sleep(0.05)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['not cancelled', 'cancelled']
print(classification_report(actual_list, predicted_list, target_names=target_names))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(actual_list, predicted_list)

In [ ]:
predictor.delete_endpoint()